# Q1

In [1]:
asset = 100
asset_c = 0.07 / 12
classA = 75
classA_c = 0.06 / 12
classB = 15
classB_c = 0.09 / 12
T = 10 * 12

## (a) 0% annualized default rate

In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

months = list(range(1,T+1))
df = pd.DataFrame({'Month':months, 'Not\'I': asset, 'Not\'I after default':asset, 'Princ':0,'Int':asset * asset_c,
                  'Class A Not\'I':classA,'Class A Int':classA*classA_c,'Class A Princ':0,'End Class A Not\'I':classA,
                  'Class B Not\'I':classB,'Class B Int':classB*classB_c,'Class B Princ':0,'End Class B Not\'I':classB,
                  'Equity Not\'I':10,'Equity Int':asset*asset_c-classA*classA_c-classB*classB_c,'Equity Princ':0,'End Equity Not\'I':10
                  })
df.loc[119, 'Princ'] = asset
df.loc[119, 'Class A Princ'] = classA
df.loc[119, 'End Class A Not\'I'] = 0
df.loc[119, 'Class B Princ'] = classB
df.loc[119, 'End Class B Not\'I'] = 0
df.loc[119, 'Equity Princ'] = 10
df.loc[119, 'End Equity Not\'I'] = 0

In [3]:
df.to_excel('Q1_a.xlsx',index=False)

## (b) 1% annualized default rate 

In [4]:
mthly_default = 1 - 0.99**(1/12)

In [5]:
Not_aft_def = [100]
Int = []
equity_Int = []
classA_int = classA * classA_c
classB_int = classB * classB_c
for i in range(120):
    Not_aft_def.append(Not_aft_def[-1] * (1-mthly_default))
    Int.append(Not_aft_def[-1] * asset_c)
    equity_Int.append(Int[-1] - classA_int - classB_int)

df = pd.DataFrame({'Month':months, 'Not\'I': Not_aft_def[:-1], 'Not\'I after default':Not_aft_def[1:], 'Princ':0,'Int':Int,
                  'Class A Not\'I':classA,'Class A Int':classA*classA_c,'Class A Princ':0,'End Class A Not\'I':classA,
                  'Class B Not\'I':classB,'Class B Int':classB*classB_c,'Class B Princ':0,'End Class B Not\'I':classB,
                  'Equity Not\'I':10,'Equity Int':equity_Int,'Equity Princ':0,'End Equity Not\'I':10
                  })

df.loc[119, 'Princ'] = Not_aft_def[-1]
df.loc[119, 'Class A Princ'] = classA if Not_aft_def[-1] >= classA else Not_aft_def[-1]
df.loc[119, 'End Class A Not\'I'] = 0 if Not_aft_def[-1] >= classA else classA - Not_aft_def[-1]
df.loc[119, 'Class B Princ'] = classB if Not_aft_def[-1] >= classA + classB else Not_aft_def[-1] - classA
df.loc[119, 'End Class B Not\'I'] = 0 if Not_aft_def[-1] >= classA + classB else classA + classB - Not_aft_def[-1]
df.loc[119, 'Equity Princ'] = max(Not_aft_def[-1] - classA - classB, 0)
df.loc[119, 'End Equity Not\'I'] = 10 - max(Not_aft_def[-1] - classA - classB, 0)
df.to_excel('Q1_b.xlsx',index=False)

## (c)/(d)

In [6]:
import scipy.optimize as opt

def solve_default(mthly_default):
    return ((1-mthly_default)**120*100000000 - 89999999)**2

soln = opt.minimize_scalar(solve_default, method='bounded', bounds=(0, 0.01))
print('Annualized default rate that causes the B class to lose ~$1 of principal: ', 1 - (1-soln.x)**12)
def solve_default2(mthly_default):
    return ((1-mthly_default)**120*100000000 - 74999999)**2

soln = opt.minimize_scalar(solve_default2, method='bounded', bounds=(0, 0.01))
print('Annualized default rate that causes the A class to lose ~$1 of principal: ', 1 - (1-soln.x)**12)

Annualized default rate that causes the B class to lose ~$1 of principal:  0.010479423841461277
Annualized default rate that causes the A class to lose ~$1 of principal:  0.02836064622292489


# Q2

In [7]:
triggerA = 1.2
triggerB = 1.05

## (a) 0% annualized default rate

When there is no default, the answer is the same as without OC test.

## (b) 1% annualized default rate 

In [8]:
Not_aft_def = [100.]
Int = []
classA_Not = [75.]
classA_int = []
classA_princ = []
classB_Not = [15.]
classB_int = []
classB_princ = []

equity_Int = []

for i in range(120):
    Not_aft_def.append(Not_aft_def[-1] * (1-mthly_default))
    Int.append(Not_aft_def[-1] * asset_c)
    ratio_A = Not_aft_def[-1] / classA_Not[-1]
    if ratio_A >= triggerA:
        classA_int.append(classA_Not[-1] * classA_c)
        classA_princ.append(0)
        classA_Not.append(classA_Not[-1])
        ratio_B = Not_aft_def[-1] / (classA_Not[-1]+classB_Not[-1])
        if ratio_B >= triggerB:
            classB_int.append(classB_Not[-1] * classB_c)
            classB_princ.append(0)
            classB_Not.append(classB_Not[-1])
            equity_Int.append(Int[-1]-classA_int[-1]-classB_int[-1])
        else:
            classB_int.append(classB_Not[-1] * classB_c)
            classB_princ.append(min(classA_Not[-1]+classB_Not[-1]-Not_aft_def[-1]/triggerB,Int[-1]-classA_int[-1]-classB_int[-1]))
            classB_Not.append(classB_Not[-1] - classB_princ[-1])
            equity_Int.append(max(Int[-1]-classA_int[-1]-classB_int[-1]-classB_princ[-1],0))
    else:
        classA_int.append(classA_Not[-1] * classA_c)
        classA_princ.append(min(classA_Not[-1]-Not_aft_def[-1]/triggerA,Int[-1]-classA_int[-1]))
        classA_Not.append(classA_Not[-1]-classA_princ[-1])
        ratio_B = Not_aft_def[-1] / (classA_Not[-1]+classB_Not[-1])
        if ratio_B >= triggerB:
            classB_int.append(min(classB_Not[-1] * classB_c, Int[-1]-classA_int[-1]-classA_princ[-1]))
            classB_princ.append(0)
            classB_Not.append(classB_Not[-1])
            equity_Int.append(max(Int[-1]-classA_int[-1]-classA_princ[-1]-classB_int[-1],0))
        else:
            classB_int.append(min(classB_Not[-1] * classB_c, Int[-1]-classA_int[-1]-classA_princ[-1]))
            classB_princ.append(min(classA_Not[-1]+classB_Not[-1]-Not_aft_def[-1]/triggerB,Int[-1]-classA_int[-1]-classA_princ[-1]-classB_int[-1]))
            classB_Not.append(classB_Not[-1] - classB_princ[-1])
            equity_Int.append(max(Int[-1]-classA_int[-1]-classA_princ[-1]-classB_int[-1]-classB_princ[-1],0))     
        

In [9]:
df = pd.DataFrame({'Month':months, 'Not\'I': Not_aft_def[:-1], 'Not\'I after default':Not_aft_def[1:], 'Princ':0,'Int':Int,
                  'Class A Not\'I':classA_Not[:-1],'Class A Int':classA_int,'Class A Princ':classA_princ,'End Class A Not\'I':classA_Not[1:],
                  'Class B Not\'I':classB_Not[:-1],'Class B Int':classB_int,'Class B Princ':classB_princ,'End Class B Not\'I':classB_Not[1:],
                  'Equity Not\'I':10,'Equity Int':equity_Int,'Equity Princ':0,'End Equity Not\'I':10
                  })

df.loc[119, 'Princ'] = Not_aft_def[-1]
df.loc[119, 'Class A Princ'] = classA_Not[-1] if Not_aft_def[-1] >= classA_Not[-1] else Not_aft_def[-1]
df.loc[119, 'End Class A Not\'I'] = 0 if Not_aft_def[-1] >= classA_Not[-1] else classA_Not[-1] - Not_aft_def[-1]
df.loc[119, 'Class B Princ'] = classB_Not[-1] if Not_aft_def[-1] >= classA_Not[-1] + classB_Not[-1] else max(Not_aft_def[-1] - classA_Not[-1],0)
df.loc[119, 'End Class B Not\'I'] = classB - df['Class B Princ'].sum()
df.loc[119, 'Equity Princ'] = max(Not_aft_def[-1] - classA_Not[-1] - classB_Not[-1], 0)
df.loc[119, 'End Equity Not\'I'] = 10 - max(Not_aft_def[-1] - classA_Not[-1] - classB_Not[-1], 0)
df.to_excel('Q2_b.xlsx',index=False)

## (c)/(d)

In [10]:
def solve_default3(mthly_default):
    Not_aft_def = [100.]
    Int = []
    classA_Not = [75.]
    classA_int = []
    classA_princ = []
    classB_Not = [15.]
    classB_int = []
    classB_princ = []

    equity_Int = []

    for i in range(120):
        Not_aft_def.append(Not_aft_def[-1] * (1-mthly_default))
        Int.append(Not_aft_def[-1] * asset_c)
        ratio_A = Not_aft_def[-1] / classA_Not[-1]
        if ratio_A >= triggerA:
            classA_int.append(classA_Not[-1] * classA_c)
            classA_princ.append(0)
            classA_Not.append(classA_Not[-1])
            ratio_B = Not_aft_def[-1] / (classA_Not[-1]+classB_Not[-1])
            if ratio_B >= triggerB:
                classB_int.append(classB_Not[-1] * classB_c)
                classB_princ.append(0)
                classB_Not.append(classB_Not[-1])
                equity_Int.append(Int[-1]-classA_int[-1]-classB_int[-1])
            else:
                classB_int.append(classB_Not[-1] * classB_c)
                classB_princ.append(min(classA_Not[-1]+classB_Not[-1]-Not_aft_def[-1]/triggerB,Int[-1]-classA_int[-1]-classB_int[-1]))
                classB_Not.append(classB_Not[-1] - classB_princ[-1])
                equity_Int.append(max(Int[-1]-classA_int[-1]-classB_int[-1]-classB_princ[-1],0))
        else:
            classA_int.append(classA_Not[-1] * classA_c)
            classA_princ.append(min(classA_Not[-1]-Not_aft_def[-1]/triggerA,Int[-1]-classA_int[-1]))
            classA_Not.append(classA_Not[-1]-classA_princ[-1])
            ratio_B = Not_aft_def[-1] / (classA_Not[-1]+classB_Not[-1])
            if ratio_B >= triggerB:
                classB_int.append(min(classB_Not[-1] * classB_c, Int[-1]-classA_int[-1]-classA_princ[-1]))
                classB_princ.append(0)
                classB_Not.append(classB_Not[-1])
                equity_Int.append(max(Int[-1]-classA_int[-1]-classA_princ[-1]-classB_int[-1],0))
            else:
                classB_int.append(min(classB_Not[-1] * classB_c, Int[-1]-classA_int[-1]-classA_princ[-1]))
                classB_princ.append(min(classA_Not[-1]+classB_Not[-1]-Not_aft_def[-1]/triggerB,Int[-1]-classA_int[-1]-classA_princ[-1]-classB_int[-1]))
                classB_Not.append(classB_Not[-1] - classB_princ[-1])
                equity_Int.append(max(Int[-1]-classA_int[-1]-classA_princ[-1]-classB_int[-1]-classB_princ[-1],0))  
                
    return abs((Not_aft_def[-1]-classA_Not[-1]-classB_Not[-1])*1000000+1)

In [11]:
def solve_default4(mthly_default):
    Not_aft_def = [100.]
    Int = []
    classA_Not = [75.]
    classA_int = []
    classA_princ = []
    classB_Not = [15.]
    classB_int = []
    classB_princ = []

    equity_Int = []

    for i in range(120):
        Not_aft_def.append(Not_aft_def[-1] * (1-mthly_default))
        Int.append(Not_aft_def[-1] * asset_c)
        ratio_A = Not_aft_def[-1] / classA_Not[-1]
        if ratio_A >= triggerA:
            classA_int.append(classA_Not[-1] * classA_c)
            classA_princ.append(0)
            classA_Not.append(classA_Not[-1])
            ratio_B = Not_aft_def[-1] / (classA_Not[-1]+classB_Not[-1])
            if ratio_B >= triggerB:
                classB_int.append(classB_Not[-1] * classB_c)
                classB_princ.append(0)
                classB_Not.append(classB_Not[-1])
                equity_Int.append(Int[-1]-classA_int[-1]-classB_int[-1])
            else:
                classB_int.append(classB_Not[-1] * classB_c)
                classB_princ.append(min(classA_Not[-1]+classB_Not[-1]-Not_aft_def[-1]/triggerB,Int[-1]-classA_int[-1]-classB_int[-1]))
                classB_Not.append(classB_Not[-1] - classB_princ[-1])
                equity_Int.append(max(Int[-1]-classA_int[-1]-classB_int[-1]-classB_princ[-1],0))
        else:
            classA_int.append(classA_Not[-1] * classA_c)
            classA_princ.append(min(classA_Not[-1]-Not_aft_def[-1]/triggerA,Int[-1]-classA_int[-1]))
            classA_Not.append(classA_Not[-1]-classA_princ[-1])
            ratio_B = Not_aft_def[-1] / (classA_Not[-1]+classB_Not[-1])
            if ratio_B >= triggerB:
                classB_int.append(min(classB_Not[-1] * classB_c, Int[-1]-classA_int[-1]-classA_princ[-1]))
                classB_princ.append(0)
                classB_Not.append(classB_Not[-1])
                equity_Int.append(max(Int[-1]-classA_int[-1]-classA_princ[-1]-classB_int[-1],0))
            else:
                classB_int.append(min(classB_Not[-1] * classB_c, Int[-1]-classA_int[-1]-classA_princ[-1]))
                classB_princ.append(min(classA_Not[-1]+classB_Not[-1]-Not_aft_def[-1]/triggerB,Int[-1]-classA_int[-1]-classA_princ[-1]-classB_int[-1]))
                classB_Not.append(classB_Not[-1] - classB_princ[-1])
                equity_Int.append(max(Int[-1]-classA_int[-1]-classA_princ[-1]-classB_int[-1]-classB_princ[-1],0))  
                
    return abs((Not_aft_def[-1]-classA_Not[-1])*1000000+1)

In [12]:
soln = opt.minimize_scalar(solve_default3, method='bounded', bounds=(0.001, 0.02))
print('Annualized default rate that causes the B class to lose ~$1 of principal: ', 1 - (1-soln.x)**12)
soln = opt.minimize_scalar(solve_default4, method='bounded', bounds=(0, 0.1))
print('Annualized default rate that causes the A class to lose ~$1 of principal: ', 1 - (1-soln.x)**12)

Annualized default rate that causes the B class to lose ~$1 of principal:  0.021256358687119836
Annualized default rate that causes the A class to lose ~$1 of principal:  0.04089217942572998
